In [ ]:
pip install gurobipy numpy

In [ ]:
from gurobipy import Model, GRB, quicksum
import numpy as np
import random
from dataclasses import dataclass
from typing import List, Dict, Tuple

@dataclass
class TBRDInstance:
    """Class to hold a TBRD problem instance"""
    C: List[int]  # Customers
    D: List[int]  # Drop-off points
    R: List[int]  # Robot depots
    deadlines: Dict[int, float]  # Customer deadlines
    weights: Dict[int, float]    # Customer weights
    initial_robots: int          # Number of robots initially loaded (δ)
    truck_capacity: int          # Maximum loading capacity for robots (K)
    truck_travel_times: Dict[Tuple[str, str], float]  # Travel times for truck
    robot_travel_times: Dict[Tuple[str, int], float]  # Travel times for robots
    coordinates: Dict[str, Tuple[float, float]]       # Coordinates of all points

class TBRDInstanceGenerator:
    def __init__(self, 
                 w: float,                # Square side length (km)
                 num_customers: int,      # Number of customers
                 num_dropoff: int,        # Number of drop-off points
                 num_depots: int,         # Number of robot depots
                 truck_speed: float,      # Truck speed (km/h)
                 robot_speed: float,      # Robot speed (km/h)
                 truck_capacity: int,     # Truck capacity (K)
                 rho_min: float,          # Minimum deadline factor
                 rho_max: float,          # Maximum deadline factor
                 w_min: float,            # Minimum customer weight
                 w_max: float):           # Maximum customer weight
        self.w = w
        self.num_customers = num_customers
        self.num_dropoff = num_dropoff
        self.num_depots = num_depots
        self.truck_speed = truck_speed
        self.robot_speed = robot_speed
        self.truck_capacity = truck_capacity
        self.rho_min = rho_min
        self.rho_max = rho_max
        self.w_min = w_min
        self.w_max = w_max
        self.grid_size = 1/6

    def generate_instance(self, seed: int = None) -> TBRDInstance:
        """Generate a single TBRD instance"""
        if seed is not None:
            np.random.seed(seed)
            random.seed(seed)
            
        # Generate coordinates for all points
        coordinates = self._generate_layout()
        
        # Calculate travel times
        truck_travel_times = self._calculate_truck_travel_times(coordinates)
        robot_travel_times = self._calculate_robot_travel_times(coordinates)
        
        # Generate customer deadlines
        deadlines = self._generate_deadlines(coordinates, truck_travel_times, robot_travel_times)
        
        # Generate customer weights
        weights = self._generate_weights()
        
        # Create customer, drop-off, and depot lists
        C = list(range(self.num_customers))
        D = list(range(self.num_dropoff))
        R = list(range(self.num_depots))
        
        return TBRDInstance(
            C=C,
            D=D,
            R=R,
            deadlines=deadlines,
            weights=weights,
            initial_robots=self.truck_capacity,  # δ = K as specified
            truck_capacity=self.truck_capacity,
            truck_travel_times=truck_travel_times,
            robot_travel_times=robot_travel_times,
            coordinates=coordinates
        )

    def _generate_layout(self) -> Dict[str, Tuple[float, float]]:
        """Generate geographical layout of depots, drop-off points, and customers"""
        coordinates = {}
        
        # Generate equidistant robot depots
        depots_per_side = int(np.ceil(np.sqrt(self.num_depots)))
        spacing = self.w / (depots_per_side + 1)
        
        depot_idx = 0
        for i in range(1, depots_per_side + 1):
            for j in range(1, depots_per_side + 1):
                if depot_idx < self.num_depots:
                    coordinates[f'R{depot_idx}'] = (i * spacing, j * spacing)
                    depot_idx += 1
        
        # Generate random drop-off points
        used_positions = set((x, y) for x, y in coordinates.values())
        for i in range(self.num_dropoff):
            while True:
                x = random.uniform(0, self.w)
                y = random.uniform(0, self.w)
                pos = (round(x/self.grid_size)*self.grid_size, 
                      round(y/self.grid_size)*self.grid_size)
                if pos not in used_positions:
                    coordinates[f'D{i}'] = pos
                    used_positions.add(pos)
                    break
        
        # Generate random customer positions
        for i in range(self.num_customers):
            while True:
                x = random.uniform(0, self.w)
                y = random.uniform(0, self.w)
                pos = (round(x/self.grid_size)*self.grid_size, 
                      round(y/self.grid_size)*self.grid_size)
                if pos not in used_positions:
                    coordinates[f'C{i}'] = pos
                    used_positions.add(pos)
                    break
        
        # Set initial truck position (γ)
        possible_starts = [k for k in coordinates.keys() if k.startswith(('R', 'D'))]
        gamma = random.choice(possible_starts)
        coordinates['gamma'] = coordinates[gamma]
        
        return coordinates

    def _calculate_euclidean_distance(self, p1: Tuple[float, float], 
                                    p2: Tuple[float, float]) -> float:
        """Calculate Euclidean distance between two points"""
        return np.sqrt((p1[0] - p2[0])**2 + (p1[1] - p2[1])**2)

    def _calculate_truck_travel_times(self, 
                                    coordinates: Dict[str, Tuple[float, float]],
                                    mu_t: float = 0.1  # Adding truck handling time parameter (e.g., 0.1 hours)
                                    ) -> Dict[Tuple[str, str], float]:
        """Calculate truck travel times between all relevant points with handling times"""
        travel_times = {}
        locations = [k for k in coordinates.keys() if k != 'gamma_e']
    
        for i in locations:
            for j in locations:
                if i != j:
                    # Base travel time using Euclidean distance
                    distance = self._calculate_euclidean_distance(
                        coordinates[i], coordinates[j])
                    base_time = distance / self.truck_speed
                
                    # Add handling time for stops
                    travel_times[(i, j)] = base_time + mu_t
    
        return travel_times

    def _calculate_robot_travel_times(self, 
                                    coordinates: Dict[str, Tuple[float, float]],
                                    mu_r: float = 0.05  # Adding robot handling time parameter (e.g., 0.05 hours)
                                    ) -> Dict[Tuple[str, int], float]:
        """Calculate robot travel times including handling and return times"""
        travel_times = {}
        launch_points = [k for k in coordinates.keys() 
                        if k.startswith(('R', 'D', 'gamma'))]
    
        for i in launch_points:
            for k in range(self.num_customers):
                customer = f'C{k}'
                # Calculate one-way distance
                distance = self._calculate_euclidean_distance(
                    coordinates[i], coordinates[customer])
            
                # Account for:
                # 1. Travel time to customer
                # 2. Robot handling time at launch
                # 3. Robot handling time at customer (for return)
                # 4. Return journey (if required)
                one_way_time = distance / self.robot_speed
                travel_times[(i, k)] = one_way_time + 2 * mu_r  # Adding handling times
    
        return travel_times

    def _generate_deadlines(self, 
                          coordinates: Dict[str, Tuple[float, float]],
                          truck_times: Dict[Tuple[str, str], float],
                          robot_times: Dict[Tuple[str, int], float]
                          ) -> Dict[int, float]:
        """Generate customer deadlines based on minimum travel times"""
        deadlines = {}
        
        for k in range(self.num_customers):
            # Calculate minimum travel time to customer k
            min_time = float('inf')
            for j in [p for p in coordinates.keys() if p.startswith(('R', 'D'))]:
                if ('gamma', j) in truck_times and (j, k) in robot_times:
                    total_time = truck_times[('gamma', j)] + robot_times[(j, k)]
                    min_time = min(min_time, total_time)
            
            # Generate random factor and calculate deadline
            r_k = random.uniform(0, 1)
            deadlines[k] = min_time * (self.rho_min + (self.rho_max - self.rho_min) * r_k)
            
        return deadlines

    def _generate_weights(self) -> Dict[int, float]:
        """Generate random customer weights"""
        return {k: random.uniform(self.w_min, self.w_max) 
                for k in range(self.num_customers)}

class TBRDModel:
    def __init__(self, instance: TBRDInstance):
        """Initialize TBRD model with problem instance"""
        self.instance = instance
        self.gamma = 'gamma'
        self.gamma_e = 'gamma_e'
        
        # Create string identifiers for locations
        self.depot_names = [f'R{i}' for i in self.instance.R]
        self.dropoff_names = [f'D{i}' for i in self.instance.D]
        
        # Calculate big M based on problem parameters
        self.M = self._calculate_big_M()
        
        # Create Gurobi model
        self.model = Model("TBRD")
        
        self._create_variables()
        self._add_constraints()
        self._set_objective()

    def _calculate_big_M(self) -> float:
        """Calculate appropriate Big M value based on problem parameters"""
        max_travel_time = max(self.instance.truck_travel_times.values(), default=0)
        max_deadline = max(self.instance.deadlines.values(), default=0)
        return max(max_travel_time, max_deadline) * 2

    def _create_variables(self):
        """Create the decision variables for the model"""
        # l[j]: Amount of robots on board at departure from location j
        self.l = self.model.addVars(
            self.dropoff_names + ['gamma'],
            lb=0, name="l"
        )
        
        # s[i,j]: 1 if location j is direct successor of i
        locations = self.dropoff_names + self.depot_names + ['gamma']
        locations_plus = self.dropoff_names + self.depot_names + ['gamma_e']
        self.s = self.model.addVars(
            [(i, j) for i in locations for j in locations_plus if i != j],
            vtype=GRB.BINARY, name="s"
        )
        
        # t[j]: Arrival time at location j
        self.t = self.model.addVars(
            self.dropoff_names + self.depot_names + ['gamma'],
            lb=0, name="t"
        )
        
        # x[j,k]: 1 if customer k is supplied from location j
        self.x = self.model.addVars(
            [(j, k) for j in (self.dropoff_names + self.depot_names + ['gamma'])
             for k in self.instance.C],
            vtype=GRB.BINARY, name="x"
        )
        
        # z[k]: 1 if customer k is supplied late
        self.z = self.model.addVars(
            self.instance.C,
            vtype=GRB.BINARY, name="z"
        )

    def _add_constraints(self):
        """Add all constraints to the model"""
        # Constraint (2): One-to-one mapping between customers and launch locations
        for k in self.instance.C:
            self.model.addConstr(
                quicksum(self.x[j,k] for j in self.dropoff_names + self.depot_names + ['gamma']) == 1
            )
        
        # Constraint (3): Initial loading
        self.model.addConstr(
            self.l['gamma'] <= self.instance.initial_robots - 
            quicksum(self.x['gamma',k] for k in self.instance.C)
        )
        
        # Constraints (4-5): Robot loading after visiting locations
        for i in self.depot_names:
            for j in self.dropoff_names:
                self.model.addConstr(
                    self.l[j] <= self.instance.truck_capacity + 
                    self.M * (1 - self.s[i,j]) -
                    quicksum(self.x[j,k] for k in self.instance.C)
                )
        
        for i in self.dropoff_names + ['gamma']:
            for j in self.dropoff_names:
                if i != j:
                    self.model.addConstr(
                        self.l[j] <= self.l[i] + 
                        self.M * (1 - self.s[i,j]) -
                        quicksum(self.x[j,k] for k in self.instance.C)
                    )
        
        # Constraints (6-7): Arrival times
        self.model.addConstr(self.t['gamma'] == 0)
        
        for i in self.dropoff_names + self.depot_names + ['gamma']:
            for j in self.dropoff_names + self.depot_names:
                if i != j:
                    self.model.addConstr(
                        self.t[j] >= self.t[i] - 
                        self.M * (1 - self.s[i,j]) +
                        self.instance.truck_travel_times.get((i,j), 0)
                    )
        
        # Constraint (8): Late delivery determination
        for j in self.dropoff_names + self.depot_names + ['gamma']:
            for k in self.instance.C:
                self.model.addConstr(
                    self.M * self.z[k] >= 
                    self.t[j] - self.M * (1 - self.x[j,k]) +
                    self.instance.robot_travel_times.get((j,k), 0) - 
                    self.instance.deadlines[k]
                )
        
        # Constraints (9-10): Subtour elimination
        self.model.addConstr(
            quicksum(self.s['gamma',j] for j in 
                    self.dropoff_names + self.depot_names + ['gamma_e']) <= 1
        )
        
        for j in self.dropoff_names + self.depot_names:
            self.model.addConstr(
                quicksum(self.s[j,i] for i in 
                        (self.dropoff_names + self.depot_names + ['gamma_e']) if i != j) ==
                quicksum(self.s[i,j] for i in 
                        (self.dropoff_names + self.depot_names + ['gamma']) if i != j)
            )
        
        # Constraints (11-13): Valid tour constraints
        for j in self.dropoff_names + self.depot_names:
            self.model.addConstr(
                quicksum(self.x[j,k] for k in self.instance.C) <=
                self.M * quicksum(self.s[i,j] for i in 
                                (self.dropoff_names + self.depot_names + ['gamma']) if i != j)
            )
            
        for j in self.dropoff_names:
            self.model.addConstr(
                quicksum(self.x[j,k] for k in self.instance.C) >=
                quicksum(self.s[i,j] for i in 
                        (self.dropoff_names + self.depot_names + ['gamma']) if i != j)
            )
            
        for j in self.depot_names:
            self.model.addConstr(
                quicksum(self.x[j,k] for k in self.instance.C) + 
                quicksum(self.s[j,i] for i in self.dropoff_names) >=
                quicksum(self.s[i,j] for i in 
                        (self.dropoff_names + self.depot_names + ['gamma']) if i != j)
            )

    def _set_objective(self):
        """Set the objective function (1)"""
        self.model.setObjective(
            quicksum(self.z[k] * self.instance.weights[k] for k in self.instance.C),
            GRB.MINIMIZE
        )

    def solve(self, time_limit: int = None) -> int:
        """Solve the model with optional time limit"""
        if time_limit:
            self.model.setParam('TimeLimit', time_limit)
            
        self.model.optimize()
        return self.model.status

    def get_solution(self) -> Dict:
        """Return the solution details if a feasible solution was found"""
        if self.model.SolCount > 0:
            solution = {
                'status': self.model.status,
                'status_text': self.model.status == GRB.OPTIMAL and "Optimal" or "Time Limit Reached",
                'objective_value': self.model.objVal,
                'best_bound': self.model.objBound,
                'gap': self.model.MIPGap,
                'late_deliveries': {k: self.z[k].X for k in self.instance.C},
                'route': self._extract_route(),
                'customer_assignments': self._extract_assignments(),
                'coordinates': self.instance.coordinates,
                'completion_time': self.model.Runtime
            }
            return solution
        return None

    def _extract_route(self) -> List[str]:
        """Extract the truck route from the solution"""
        route = []
        current = self.gamma
        while current != self.gamma_e:
            route.append(current)
            for j in self.dropoff_names + self.depot_names + [self.gamma_e]:
                if j != current and self.s[current,j].X > 0.5:
                    current = j
                    break
        return route

    def _extract_assignments(self) -> Dict[int, str]:
        """Extract customer-to-location assignments from the solution"""
        assignments = {}
        for k in self.instance.C:
            for j in self.dropoff_names + self.depot_names + ['gamma']:
                if self.x[j,k].X > 0.5:
                    assignments[k] = j
        return assignments

def check_feasibility(instance):
    """Analyze why an instance might be infeasible"""
    print("\nFeasibility Analysis:")
    min_delivery_times = {}
    
    # Check delivery time feasibility
    for k in range(len(instance.C)):  # Changed from instance.num_customers
        min_time = float('inf')
        customer_pos = instance.coordinates[f'C{k}']
        
        # Check all possible service points
        for j in [p for p in instance.coordinates.keys() if p.startswith(('R', 'D', 'gamma'))]:
            service_pos = instance.coordinates[j]
            if ('gamma', j) in instance.truck_travel_times and (j, k) in instance.robot_travel_times:
                total_time = instance.truck_travel_times[('gamma', j)] + instance.robot_travel_times[(j, k)]
                min_time = min(min_time, total_time)
        
        min_delivery_times[k] = min_time
        deadline = instance.deadlines[k]
        print(f"\nCustomer {k}:")
        print(f"  Position: {instance.coordinates[f'C{k}']}")
        print(f"  Minimum possible delivery time: {min_time:.2f}")
        print(f"  Deadline: {deadline:.2f}")
        if min_time > deadline:
            print(f"  WARNING: Cannot be served within deadline!")
    
    # Check capacity-related constraints
    num_customers = len(instance.C)  # Changed from instance.num_customers
    truck_capacity = instance.truck_capacity
    print(f"\nCapacity Analysis:")
    print(f"Number of customers to serve: {num_customers}")
    print(f"Truck capacity (robots): {truck_capacity}")
    if num_customers > truck_capacity:
        print("Note: Multiple trips to depots will be needed for robot replenishment")
    
    return min_delivery_times

def run_experiment(instance_params: Dict, 
                  num_instances: int = 1,
                  time_limit: int = None) -> List[Dict]:
    """Run experiments with specified parameters"""
    results = []
    generator = TBRDInstanceGenerator(**instance_params)
    
    for i in range(num_instances):
        print(f"\nSolving instance {i+1}/{num_instances}")
        
        # Generate instance
        instance = generator.generate_instance(seed=i)
        
        # Create and solve model
        model = TBRDModel(instance)
        status = model.solve(time_limit=time_limit)
        
        # Get solution
        solution = model.get_solution()
        
        if solution:
            solution['instance_id'] = i
            solution['status'] = status
            results.append(solution)
            print(f"Status: {status}")
            print(f"Objective value: {solution['objective_value']:.2f}")
            print(f"Completion time: {solution['completion_time']:.2f} seconds")
        else:
            print(f"No solution found for instance {i+1}")
            # Run feasibility analysis when instance is infeasible
            min_delivery_times = check_feasibility(instance)
    
    return results

# [Rest of the main script remains the same]

if __name__ == "__main__":
    # Parameters remain the same
    instance_params = {
        'w': 2,                # 2 km square area
        'num_customers': 6,    # Number of customers
        'num_dropoff': 6,      # Number of drop-off points
        'num_depots': 4,       # Number of robot depots
        'truck_speed': 30,     # 30 km/h
        'robot_speed': 5,      # 5 km/h
        'truck_capacity': 2,   # Maximum of 2 robots
        'rho_min': 3.0,        # Minimum deadline factor of 3
        'rho_max': 5.0,        # Maximum deadline factor of 5
        'w_min': 1,           # Fixed weight of 1
        'w_max': 1            # Fixed weight of 1
    }
    
    results = run_experiment(
        instance_params,
        num_instances=5,
        time_limit=1800
    )
    
    if results:
        # Only consider feasible solutions
        feasible_results = [r for r in results if r['status'] == GRB.OPTIMAL]
        
        if feasible_results:
            # Find the best (minimum) objective value across feasible solutions
            min_objective = min(r['objective_value'] for r in feasible_results)
            
            # Count solutions matching the best objective value
            best_solutions = [r for r in feasible_results if r['objective_value'] == min_objective]
            num_best_solutions = len(best_solutions)
            
            # Count proven optimal solutions among the best solutions
            proven_optimal = sum(1 for r in best_solutions if r['status'] == GRB.OPTIMAL)
            
            # Calculate average CPU time for best solutions
            avg_time_best = sum(r['completion_time'] for r in best_solutions) / num_best_solutions
            
            print("\nPerformance Metrics:")
            print(f"Number of instances with best objective value ({min_objective}): {num_best_solutions}")
            print(f"Number of proven optimal solutions among best: {proven_optimal}")
            print(f"Average CPU time for best solutions: {avg_time_best:.2f} seconds")
            
            print("\nDetailed Results:")
            for i in range(5):  # For all 5 instances
                print(f"\nInstance {i+1}:")
                instance_results = [r for r in results if r['instance_id'] == i]
                if instance_results and instance_results[0]['status'] == GRB.OPTIMAL:
                    r = instance_results[0]
                    print(f"Status: Optimal")
                    print(f"Objective value: {r['objective_value']:.2f}")
                    print(f"Solution time: {r['completion_time']:.2f} seconds")
                    print(f"Route: {r['route']}")
                    print(f"Customer assignments: {r['customer_assignments']}")
                else:
                    print(f"Status: Infeasible")
        else:
            print("\nNo feasible solutions found in any instance.")

The feasibility analysis now helps us understand why instances 1 and 4 are infeasible. Let's analyze both:

Instance 1:
- All customers can theoretically be reached within their deadlines (minimum delivery times are less than deadlines for all customers)
- For example, Customer 1 has:
  - Minimum delivery time: 0.05 hours
  - Deadline: 0.20 hours
- The issue likely comes from the truck capacity constraint:
  - Need to serve 6 customers
  - Truck can only carry 2 robots
  - Need multiple trips to depots
  - These multiple trips might make it impossible to serve all customers within their deadlines

Instance 4:
- Similar situation to Instance 1
- All individual customers could theoretically be served within their deadlines
- For example, Customer 0 has:
  - Minimum delivery time: 0.12 hours
  - Deadline: 0.46 hours
- Again, the combination of:
  - Limited truck capacity (2 robots)
  - Number of customers (6)
  - Need for multiple depot visits
Makes it impossible to find a feasible route that serves all customers on time.

For both infeasible instances, it's not that any single customer is impossible to serve, but rather that the combination of capacity constraints and tight deadlines makes it impossible to serve all customers within their deadlines when multiple trips are required.

- So was it just random chance that the 3 feasible solutions found happened to have a combination of parameters which could have a feasible solution??
- How are optimal solutions being found in such a short time when the paper says the average time taken to solve each instance was over 400 sec.

In [ ]:
from gurobipy import Model, GRB, quicksum, Env as gp
import numpy as np
import random
from dataclasses import dataclass
from typing import List, Dict, Tuple

@dataclass
class TBRDInstance:
    """Class to hold a TBRD problem instance"""
    C: List[int]  # Customers
    D: List[int]  # Drop-off points
    R: List[int]  # Robot depots
    deadlines: Dict[int, float]  # Customer deadlines
    weights: Dict[int, float]    # Customer weights
    initial_robots: int          # Number of robots initially loaded (δ)
    truck_capacity: int          # Maximum loading capacity for robots (K)
    truck_travel_times: Dict[Tuple[str, str], float]  # Travel times for truck
    robot_travel_times: Dict[Tuple[str, int], float]  # Travel times for robots
    coordinates: Dict[str, Tuple[float, float]]       # Coordinates of all points

class TBRDInstanceGenerator:
    def __init__(self, 
                 w: float,                # Square side length (km)
                 num_customers: int,      # Number of customers
                 num_dropoff: int,        # Number of drop-off points
                 num_depots: int,         # Number of robot depots
                 truck_speed: float,      # Truck speed (km/h)
                 robot_speed: float,      # Robot speed (km/h)
                 truck_capacity: int,     # Truck capacity (K)
                 rho_min: float,          # Minimum deadline factor
                 rho_max: float,          # Maximum deadline factor
                 w_min: float,            # Minimum customer weight
                 w_max: float):           # Maximum customer weight
        self.w = w
        self.num_customers = num_customers
        self.num_dropoff = num_dropoff
        self.num_depots = num_depots
        self.truck_speed = truck_speed
        self.robot_speed = robot_speed
        self.truck_capacity = truck_capacity
        self.rho_min = rho_min
        self.rho_max = rho_max
        self.w_min = w_min
        self.w_max = w_max
        self.grid_size = 1/6

    def generate_instance(self, seed: int = None) -> TBRDInstance:
        """Generate a single TBRD instance"""
        if seed is not None:
            np.random.seed(seed)
            random.seed(seed)
            
        # Generate coordinates for all points
        coordinates = self._generate_layout()
        
        # Calculate travel times
        truck_travel_times = self._calculate_truck_travel_times(coordinates)
        robot_travel_times = self._calculate_robot_travel_times(coordinates)
        
        # Generate customer deadlines
        deadlines = self._generate_deadlines(coordinates, truck_travel_times, robot_travel_times)
        
        # Generate customer weights
        weights = self._generate_weights()
        
        # Create customer, drop-off, and depot lists
        C = list(range(self.num_customers))
        D = list(range(self.num_dropoff))
        R = list(range(self.num_depots))
        
        return TBRDInstance(
            C=C,
            D=D,
            R=R,
            deadlines=deadlines,
            weights=weights,
            initial_robots=self.truck_capacity,  # δ = K as specified
            truck_capacity=self.truck_capacity,
            truck_travel_times=truck_travel_times,
            robot_travel_times=robot_travel_times,
            coordinates=coordinates
        )

    def _generate_layout(self) -> Dict[str, Tuple[float, float]]:
        """Generate geographical layout of depots, drop-off points, and customers"""
        coordinates = {}
        
        # Generate equidistant robot depots
        depots_per_side = int(np.ceil(np.sqrt(self.num_depots)))
        spacing = self.w / (depots_per_side + 1)
        
        depot_idx = 0
        for i in range(1, depots_per_side + 1):
            for j in range(1, depots_per_side + 1):
                if depot_idx < self.num_depots:
                    coordinates[f'R{depot_idx}'] = (i * spacing, j * spacing)
                    depot_idx += 1
        
        # Generate random drop-off points
        used_positions = set((x, y) for x, y in coordinates.values())
        for i in range(self.num_dropoff):
            while True:
                x = random.uniform(0, self.w)
                y = random.uniform(0, self.w)
                pos = (round(x/self.grid_size)*self.grid_size, 
                      round(y/self.grid_size)*self.grid_size)
                if pos not in used_positions:
                    coordinates[f'D{i}'] = pos
                    used_positions.add(pos)
                    break
        
        # Generate random customer positions
        for i in range(self.num_customers):
            while True:
                x = random.uniform(0, self.w)
                y = random.uniform(0, self.w)
                pos = (round(x/self.grid_size)*self.grid_size, 
                      round(y/self.grid_size)*self.grid_size)
                if pos not in used_positions:
                    coordinates[f'C{i}'] = pos
                    used_positions.add(pos)
                    break
        
        # Set initial truck position (γ)
        possible_starts = [k for k in coordinates.keys() if k.startswith(('R', 'D'))]
        gamma = random.choice(possible_starts)
        coordinates['gamma'] = coordinates[gamma]
        
        return coordinates

    def _calculate_euclidean_distance(self, p1: Tuple[float, float], 
                                    p2: Tuple[float, float]) -> float:
        """Calculate Euclidean distance between two points"""
        return np.sqrt((p1[0] - p2[0])**2 + (p1[1] - p2[1])**2)

    def _calculate_truck_travel_times(self, 
                                    coordinates: Dict[str, Tuple[float, float]]
                                    ) -> Dict[Tuple[str, str], float]:
        """Calculate truck travel times between all relevant points"""
        travel_times = {}
        locations = [k for k in coordinates.keys() if k != 'gamma_e']
        
        for i in locations:
            for j in locations:
                if i != j:
                    distance = self._calculate_euclidean_distance(
                        coordinates[i], coordinates[j])
                    travel_times[(i, j)] = distance / self.truck_speed
        
        return travel_times

    def _calculate_robot_travel_times(self, 
                                    coordinates: Dict[str, Tuple[float, float]]
                                    ) -> Dict[Tuple[str, int], float]:
        """Calculate robot travel times from all possible launch points to customers"""
        travel_times = {}
        launch_points = [k for k in coordinates.keys() if k.startswith(('R', 'D', 'gamma'))]
        
        for i in launch_points:
            for k in range(self.num_customers):
                customer = f'C{k}'
                distance = self._calculate_euclidean_distance(
                    coordinates[i], coordinates[customer])
                travel_times[(i, k)] = distance / self.robot_speed
        
        return travel_times

    def _generate_deadlines(self, 
                          coordinates: Dict[str, Tuple[float, float]],
                          truck_times: Dict[Tuple[str, str], float],
                          robot_times: Dict[Tuple[str, int], float]
                          ) -> Dict[int, float]:
        """Generate customer deadlines based on minimum travel times"""
        deadlines = {}
        
        for k in range(self.num_customers):
            # Calculate minimum travel time to customer k
            min_time = float('inf')
            for j in [p for p in coordinates.keys() if p.startswith(('R', 'D'))]:
                if ('gamma', j) in truck_times and (j, k) in robot_times:
                    total_time = truck_times[('gamma', j)] + robot_times[(j, k)]
                    min_time = min(min_time, total_time)
            
            # Generate random factor and calculate deadline
            r_k = random.uniform(0, 1)
            deadlines[k] = min_time * (self.rho_min + (self.rho_max - self.rho_min) * r_k)
            
        return deadlines

    def _generate_weights(self) -> Dict[int, float]:
        """Generate random customer weights"""
        return {k: random.uniform(self.w_min, self.w_max) 
                for k in range(self.num_customers)}

class TBRDModel:
    def __init__(self, instance: TBRDInstance):
        """Initialize TBRD model with problem instance"""
        self.instance = instance
        self.gamma = 'gamma'
        self.gamma_e = 'gamma_e'
    
        # Create string identifiers for locations
        self.depot_names = [f'R{i}' for i in self.instance.R]
        self.dropoff_names = [f'D{i}' for i in self.instance.D]
    
        # Calculate big M based on problem parameters
        self.M = self._calculate_big_M()
    
        # Create Gurobi model with output suppressed
        self.model = Model("TBRD")
        self.model.setParam('OutputFlag', 0)  # Suppress Gurobi output
        
        self._create_variables()
        self._add_constraints()
        self._set_objective()

    def _calculate_big_M(self) -> float:
        """Calculate appropriate Big M value based on problem parameters"""
        max_travel_time = max(self.instance.truck_travel_times.values(), default=0)
        max_deadline = max(self.instance.deadlines.values(), default=0)
        return max(max_travel_time, max_deadline) * 2

    def _create_variables(self):
        """Create the decision variables for the model"""
        # l[j]: Amount of robots on board at departure from location j
        self.l = self.model.addVars(
            self.dropoff_names + ['gamma'],
            lb=0, name="l"
        )
        
        # s[i,j]: 1 if location j is direct successor of i
        locations = self.dropoff_names + self.depot_names + ['gamma']
        locations_plus = self.dropoff_names + self.depot_names + ['gamma_e']
        self.s = self.model.addVars(
            [(i, j) for i in locations for j in locations_plus if i != j],
            vtype=GRB.BINARY, name="s"
        )
        
        # t[j]: Arrival time at location j
        self.t = self.model.addVars(
            self.dropoff_names + self.depot_names + ['gamma'],
            lb=0, name="t"
        )
        
        # x[j,k]: 1 if customer k is supplied from location j
        self.x = self.model.addVars(
            [(j, k) for j in (self.dropoff_names + self.depot_names + ['gamma'])
             for k in self.instance.C],
            vtype=GRB.BINARY, name="x"
        )
        
        # z[k]: 1 if customer k is supplied late
        self.z = self.model.addVars(
            self.instance.C,
            vtype=GRB.BINARY, name="z"
        )

    def _add_constraints(self):
        """Add all constraints to the model"""
        # Constraint (2): One-to-one mapping between customers and launch locations
        for k in self.instance.C:
            self.model.addConstr(
                quicksum(self.x[j,k] for j in self.dropoff_names + self.depot_names + ['gamma']) == 1
            )
        
        # Constraint (3): Initial loading
        self.model.addConstr(
            self.l['gamma'] <= self.instance.initial_robots - 
            quicksum(self.x['gamma',k] for k in self.instance.C)
        )
        
        # Constraints (4-5): Robot loading after visiting locations
        for i in self.depot_names:
            for j in self.dropoff_names:
                self.model.addConstr(
                    self.l[j] <= self.instance.truck_capacity + 
                    self.M * (1 - self.s[i,j]) -
                    quicksum(self.x[j,k] for k in self.instance.C)
                )
        
        for i in self.dropoff_names + ['gamma']:
            for j in self.dropoff_names:
                if i != j:
                    self.model.addConstr(
                        self.l[j] <= self.l[i] + 
                        self.M * (1 - self.s[i,j]) -
                        quicksum(self.x[j,k] for k in self.instance.C)
                    )
        
        # Constraints (6-7): Arrival times
        self.model.addConstr(self.t['gamma'] == 0)
        
        for i in self.dropoff_names + self.depot_names + ['gamma']:
            for j in self.dropoff_names + self.depot_names:
                if i != j:
                    self.model.addConstr(
                        self.t[j] >= self.t[i] - 
                        self.M * (1 - self.s[i,j]) +
                        self.instance.truck_travel_times.get((i,j), 0)
                    )
        
        # Constraint (8): Late delivery determination
        for j in self.dropoff_names + self.depot_names + ['gamma']:
            for k in self.instance.C:
                self.model.addConstr(
                    self.M * self.z[k] >= 
                    self.t[j] - self.M * (1 - self.x[j,k]) +
                    self.instance.robot_travel_times.get((j,k), 0) - 
                    self.instance.deadlines[k]
                )
        
        # Constraints (9-10): Subtour elimination
        self.model.addConstr(
            quicksum(self.s['gamma',j] for j in 
                    self.dropoff_names + self.depot_names + ['gamma_e']) <= 1
        )
        
        for j in self.dropoff_names + self.depot_names:
            self.model.addConstr(
                quicksum(self.s[j,i] for i in 
                        (self.dropoff_names + self.depot_names + ['gamma_e']) if i != j) ==
                quicksum(self.s[i,j] for i in 
                        (self.dropoff_names + self.depot_names + ['gamma']) if i != j)
            )
        
        # Constraints (11-13): Valid tour constraints
        for j in self.dropoff_names + self.depot_names:
            self.model.addConstr(
                quicksum(self.x[j,k] for k in self.instance.C) <=
                self.M * quicksum(self.s[i,j] for i in 
                                (self.dropoff_names + self.depot_names + ['gamma']) if i != j)
            )
            
        for j in self.dropoff_names:
            self.model.addConstr(
                quicksum(self.x[j,k] for k in self.instance.C) >=
                quicksum(self.s[i,j] for i in 
                        (self.dropoff_names + self.depot_names + ['gamma']) if i != j)
            )
            
        for j in self.depot_names:
            self.model.addConstr(
                quicksum(self.x[j,k] for k in self.instance.C) + 
                quicksum(self.s[j,i] for i in self.dropoff_names) >=
                quicksum(self.s[i,j] for i in 
                        (self.dropoff_names + self.depot_names + ['gamma']) if i != j)
            )

    def _set_objective(self):
        """Set the objective function (1)"""
        self.model.setObjective(
            quicksum(self.z[k] * self.instance.weights[k] for k in self.instance.C),
            GRB.MINIMIZE
        )

    def solve(self, time_limit: int = None) -> int:
        """Solve the model with optional time limit"""
        if time_limit:
            self.model.setParam('TimeLimit', time_limit)
            
        self.model.optimize()
        return self.model.status

    def get_solution(self) -> Dict:
        """Return the solution details if a feasible solution was found"""
        if self.model.SolCount > 0:
            solution = {
                'status': self.model.status,
                'status_text': self.model.status == GRB.OPTIMAL and "Optimal" or "Time Limit Reached",
                'objective_value': self.model.objVal,
                'best_bound': self.model.objBound,
                'gap': self.model.MIPGap,
                'late_deliveries': {k: self.z[k].X for k in self.instance.C},
                'route': self._extract_route(),
                'customer_assignments': self._extract_assignments(),
                'coordinates': self.instance.coordinates,
                'completion_time': self.model.Runtime
            }
            return solution
        return None

    def _extract_route(self) -> List[str]:
        """Extract the truck route from the solution"""
        route = []
        current = self.gamma
        while current != self.gamma_e:
            route.append(current)
            for j in self.dropoff_names + self.depot_names + [self.gamma_e]:
                if j != current and self.s[current,j].X > 0.5:
                    current = j
                    break
        return route

    def _extract_assignments(self) -> Dict[int, str]:
        """Extract customer-to-location assignments from the solution"""
        assignments = {}
        for k in self.instance.C:
            for j in self.dropoff_names + self.depot_names + ['gamma']:
                if self.x[j,k].X > 0.5:
                    assignments[k] = j
        return assignments

def check_feasibility(instance):
    """Analyze why an instance might be infeasible"""
    min_delivery_times = {}
    deadlines_violated = False
    
    # Check delivery time feasibility
    for k in range(len(instance.C)):
        min_time = float('inf')
        customer_pos = instance.coordinates[f'C{k}']
        
        for j in [p for p in instance.coordinates.keys() if p.startswith(('R', 'D', 'gamma'))]:
            service_pos = instance.coordinates[j]
            if ('gamma', j) in instance.truck_travel_times and (j, k) in instance.robot_travel_times:
                total_time = instance.truck_travel_times[('gamma', j)] + instance.robot_travel_times[(j, k)]
                min_time = min(min_time, total_time)
        
        min_delivery_times[k] = min_time
        if min_time > instance.deadlines[k]:
            deadlines_violated = True
    
    return {
        'min_delivery_times': min_delivery_times,
        'deadlines_violated': deadlines_violated,
        'capacity_constrained': len(instance.C) > instance.truck_capacity
    }

def run_experiment(instance_params: Dict, 
                  num_instances: int = 1,
                  time_limit: int = None,
                  save_file: str = None) -> Dict:
    """Run experiments with specified parameters"""
    results = {
        'feasible_instances': [],
        'infeasible_instances': [],
        'summary': {
            'total_instances': num_instances,
            'num_feasible': 0,
            'num_infeasible': 0,
            'num_optimal': 0,
            'avg_solution_time': 0,
            'best_solution_time': float('inf'),
            'worst_solution_time': 0,
            'infeasibility_causes': {
                'deadlines': 0,
                'capacity': 0
            }
        }
    }
    
    print(f"\nRunning {num_instances} instances...")
    generator = TBRDInstanceGenerator(**instance_params)
    
    for i in range(num_instances):
        if (i + 1) % 10 == 0:  # Progress update every 10 instances
            print(f"Processed {i + 1}/{num_instances} instances")
            
        instance = generator.generate_instance(seed=i)
        model = TBRDModel(instance)
        status = model.solve(time_limit=time_limit)
        solution = model.get_solution()
        
        if solution:
            # Feasible solution found
            results['feasible_instances'].append({
                'instance_id': i,
                'status': status,
                'objective_value': solution['objective_value'],
                'completion_time': solution['completion_time'],
                'route': solution['route'],
                'customer_assignments': solution['customer_assignments']
            })
            results['summary']['num_feasible'] += 1
            if status == GRB.OPTIMAL:
                results['summary']['num_optimal'] += 1
                
            # Update timing statistics
            results['summary']['avg_solution_time'] += solution['completion_time']
            results['summary']['best_solution_time'] = min(
                results['summary']['best_solution_time'], 
                solution['completion_time']
            )
            results['summary']['worst_solution_time'] = max(
                results['summary']['worst_solution_time'], 
                solution['completion_time']
            )
        else:
            # Analyze infeasibility
            feasibility_check = check_feasibility(instance)
            results['infeasible_instances'].append({
                'instance_id': i,
                'analysis': feasibility_check
            })
            results['summary']['num_infeasible'] += 1
            
            if feasibility_check['deadlines_violated']:
                results['summary']['infeasibility_causes']['deadlines'] += 1
            if feasibility_check['capacity_constrained']:
                results['summary']['infeasibility_causes']['capacity'] += 1
    
    # Finalize averages
    if results['summary']['num_feasible'] > 0:
        results['summary']['avg_solution_time'] /= results['summary']['num_feasible']
    
    # Save detailed results if requested
    if save_file:
        import json
        with open(save_file, 'w') as f:
            json.dump(results, f, indent=2)
    
    return results

if __name__ == "__main__":
    instance_params = {
        'w': 2,                # 2 km square area
        'num_customers': 6,    # Number of customers
        'num_dropoff': 6,      # Number of drop-off points
        'num_depots': 4,       # Number of robot depots
        'truck_speed': 30,     # 30 km/h
        'robot_speed': 5,      # 5 km/h
        'truck_capacity': 2,   # Maximum of 2 robots
        'rho_min': 3.0,        # Minimum deadline factor of 3
        'rho_max': 5.0,        # Maximum deadline factor of 5
        'w_min': 1,           # Fixed weight of 1
        'w_max': 1            # Fixed weight of 1
    }
    
    # Run experiments and save detailed results to file
    results = run_experiment(
        instance_params,
        num_instances=50,
        time_limit=1800,
        save_file='tbrd_results.json'
    )
    
    # Print summary statistics
    print("\nExperiment Summary:")
    print(f"Total instances: {results['summary']['total_instances']}")
    print(f"Feasible instances: {results['summary']['num_feasible']}")
    print(f"Optimal solutions: {results['summary']['num_optimal']}")
    print(f"Infeasible instances: {results['summary']['num_infeasible']}")
    
    if results['summary']['num_feasible'] > 0:
        print("\nSolution Times:")
        print(f"Average: {results['summary']['avg_solution_time']:.2f} seconds")
        print(f"Best: {results['summary']['best_solution_time']:.2f} seconds")
        print(f"Worst: {results['summary']['worst_solution_time']:.2f} seconds")
    
    print("\nInfeasibility Analysis:")
    print(f"Deadline violations: {results['summary']['infeasibility_causes']['deadlines']}")
    print(f"Capacity constraints: {results['summary']['infeasibility_causes']['capacity']}")
    
    print("\nDetailed results saved to 'tbrd_results.json'")

The above has the Gurobi environment suppressed so that running 50 instances doesn't result in an obnoxiously long result screen.

Given that I have taken the exact same parameters as the paper, the massive reduction in average CPU times can be due to the following:
- Hardware differences (paper has 4 × 4.0 gigahertz, my laptop has 8 x 2.8 gigahertz [windows 7 vs windows 11])
- Gurobi version differences (paper used 7.0.2, I used 12.0)
- Python vs C# differences
- Potential oversimplification of some aspects of the problem
- accepting solutions that that authors had guardrails for

In [9]:
from gurobipy import Model, GRB, quicksum, Env as gp
import numpy as np
import random
import math
from dataclasses import dataclass
from typing import List, Dict, Tuple
import threading
import _thread
from contextlib import contextmanager

@contextmanager
def timeout(seconds):
    def raise_timeout():
        _thread.interrupt_main()
    
    timer = threading.Timer(seconds, raise_timeout)
    timer.start()
    try:
        yield
    except KeyboardInterrupt:
        raise TimeoutError(f"Timed out after {seconds} seconds")
    finally:
        timer.cancel()

@dataclass
class TBRDInstance:
    """Class to hold a TBRD problem instance"""
    C: List[int]  # Customers
    D: List[int]  # Drop-off points
    R: List[int]  # Robot depots
    deadlines: Dict[int, float]  # Customer deadlines
    weights: Dict[int, float]    # Customer weights
    initial_robots: int          # Number of robots initially loaded (δ)
    truck_capacity: int          # Maximum loading capacity for robots (K)
    truck_travel_times: Dict[Tuple[str, str], float]  # Travel times for truck
    robot_travel_times: Dict[Tuple[str, int], float]  # Travel times for robots
    coordinates: Dict[str, Tuple[float, float]]       # Coordinates of all points

class TBRDInstanceGenerator:
    def __init__(self, 
                 w: float,                # Square side length (km)
                 num_customers: int,      # Number of customers
                 num_dropoff: int,        # Number of drop-off points
                 num_depots: int,         # Number of robot depots
                 truck_speed: float,      # Truck speed (km/h)
                 robot_speed: float,      # Robot speed (km/h)
                 truck_capacity: int,     # Truck capacity (K)
                 rho_min: float,          # Minimum deadline factor
                 rho_max: float,          # Maximum deadline factor
                 w_min: float,            # Minimum customer weight
                 w_max: float):           # Maximum customer weight
        self.w = w
        self.num_customers = num_customers
        self.num_dropoff = num_dropoff
        self.num_depots = num_depots
        self.truck_speed = truck_speed
        self.robot_speed = robot_speed
        self.truck_capacity = truck_capacity
        self.rho_min = rho_min
        self.rho_max = rho_max
        self.w_min = w_min
        self.w_max = w_max
        self.grid_size = 1/6

    def generate_instance(self, seed: int = None) -> TBRDInstance:
        """Generate a single TBRD instance"""
        if seed is not None:
            np.random.seed(seed)
            random.seed(seed)
            
        # Generate coordinates for all points
        coordinates = self._generate_layout()
        
        # Calculate travel times
        truck_travel_times = self._calculate_truck_travel_times(coordinates)
        robot_travel_times = self._calculate_robot_travel_times(coordinates)
        
        # Generate customer deadlines
        deadlines = self._generate_deadlines(coordinates, truck_travel_times, robot_travel_times)
        
        # Generate customer weights
        weights = self._generate_weights()
        
        # Create customer, drop-off, and depot lists
        C = list(range(self.num_customers))
        D = list(range(self.num_dropoff))
        R = list(range(self.num_depots))
        
        return TBRDInstance(
            C=C,
            D=D,
            R=R,
            deadlines=deadlines,
            weights=weights,
            initial_robots=self.truck_capacity,  # δ = K as specified
            truck_capacity=self.truck_capacity,
            truck_travel_times=truck_travel_times,
            robot_travel_times=robot_travel_times,
            coordinates=coordinates
        )

    def _generate_layout(self) -> Dict[str, Tuple[float, float]]:
        """Generate geographical layout of depots, drop-off points, and customers"""
        coordinates = {}
        
        # Generate equidistant robot depots
        depots_per_side = int(np.ceil(np.sqrt(self.num_depots)))
        spacing = self.w / (depots_per_side + 1)
        
        depot_idx = 0
        for i in range(1, depots_per_side + 1):
            for j in range(1, depots_per_side + 1):
                if depot_idx < self.num_depots:
                    coordinates[f'R{depot_idx}'] = (i * spacing, j * spacing)
                    depot_idx += 1
        
        # Generate random drop-off points
        used_positions = set((x, y) for x, y in coordinates.values())
        for i in range(self.num_dropoff):
            while True:
                x = random.uniform(0, self.w)
                y = random.uniform(0, self.w)
                pos = (round(x/self.grid_size)*self.grid_size, 
                      round(y/self.grid_size)*self.grid_size)
                if pos not in used_positions:
                    coordinates[f'D{i}'] = pos
                    used_positions.add(pos)
                    break
        
        # Generate random customer positions
        for i in range(self.num_customers):
            while True:
                x = random.uniform(0, self.w)
                y = random.uniform(0, self.w)
                pos = (round(x/self.grid_size)*self.grid_size, 
                      round(y/self.grid_size)*self.grid_size)
                if pos not in used_positions:
                    coordinates[f'C{i}'] = pos
                    used_positions.add(pos)
                    break
        
        # Set initial truck position (γ)
        possible_starts = [k for k in coordinates.keys() if k.startswith(('R', 'D'))]
        gamma = random.choice(possible_starts)
        coordinates['gamma'] = coordinates[gamma]
        
        return coordinates

    def _calculate_euclidean_distance(self, p1: Tuple[float, float], 
                                    p2: Tuple[float, float]) -> float:
        """Calculate Euclidean distance between two points"""
        return np.sqrt((p1[0] - p2[0])**2 + (p1[1] - p2[1])**2)

    def _calculate_truck_travel_times(self, 
                                    coordinates: Dict[str, Tuple[float, float]]
                                    ) -> Dict[Tuple[str, str], float]:
        """Calculate truck travel times between all relevant points"""
        travel_times = {}
        locations = [k for k in coordinates.keys() if k != 'gamma_e']
        
        for i in locations:
            for j in locations:
                if i != j:
                    distance = self._calculate_euclidean_distance(
                        coordinates[i], coordinates[j])
                    travel_times[(i, j)] = distance / self.truck_speed
        
        return travel_times

    def _calculate_robot_travel_times(self, 
                                    coordinates: Dict[str, Tuple[float, float]]
                                    ) -> Dict[Tuple[str, int], float]:
        """Calculate robot travel times from all possible launch points to customers"""
        travel_times = {}
        launch_points = [k for k in coordinates.keys() if k.startswith(('R', 'D', 'gamma'))]
        
        for i in launch_points:
            for k in range(self.num_customers):
                customer = f'C{k}'
                distance = self._calculate_euclidean_distance(
                    coordinates[i], coordinates[customer])
                travel_times[(i, k)] = distance / self.robot_speed
        
        return travel_times

    def _generate_deadlines(self, 
                           coordinates: Dict[str, Tuple[float, float]],
                           truck_times: Dict[Tuple[str, str], float],
                           robot_times: Dict[Tuple[str, int], float]
                           ) -> Dict[int, float]:
        """Generate customer deadlines based on minimum travel times including handling times"""
        deadlines = {}
    
        for k in range(self.num_customers):
            # Calculate minimum travel time to customer k
            min_time = float('inf')
            for j in [p for p in coordinates.keys() if p.startswith(('R', 'D'))]:
                if ('gamma', j) in truck_times and (j, k) in robot_times:
                    # Total time now includes handling times that were added in travel time calculations
                    total_time = truck_times[('gamma', j)] + robot_times[(j, k)]
                    min_time = min(min_time, total_time)
        
            # Generate random factor and calculate deadline
            r_k = random.uniform(0, 1)
            deadlines[k] = min_time * (self.rho_min + (self.rho_max - self.rho_min) * r_k)
            
        return deadlines

    def _generate_weights(self) -> Dict[int, float]:
        """Generate random customer weights"""
        return {k: random.uniform(self.w_min, self.w_max) 
                for k in range(self.num_customers)}
    
@dataclass
class LocationManager:
    """Manage original and duplicate locations"""
    original_dropoffs: List[int]
    depot_names: List[str]
    dropoff_duplicates: Dict[int, List[str]]  # Maps original dropoff to its duplicates
    all_dropoff_names: List[str]  # All dropoff names including duplicates
    
    @staticmethod
    def create_duplicates(C: List[int], K: int, D: List[int]) -> 'LocationManager':
        depot_names = [f'R{i}' for i in range(len(D))]
        duplicates = {}
        all_names = []
        
        # Calculate number of duplicates needed
        num_duplicates = math.ceil(len(C)/K) + 1
        
        # Create duplicates for each drop-off point
        for d in D:
            duplicates[d] = [f'D{d}_{i}' for i in range(num_duplicates)]
            all_names.extend(duplicates[d])
            
        return LocationManager(D, depot_names, duplicates, all_names)

class TBRDModel:
    def __init__(self, instance: TBRDInstance):
            self.instance = instance
            self.gamma = 'gamma'
            self.gamma_e = 'gamma_e'
    
            # Create location manager to handle duplicates
            self.loc_manager = LocationManager.create_duplicates(
                self.instance.C,
                self.instance.truck_capacity,
                self.instance.D
            )
    
            self.depot_names = self.loc_manager.depot_names
            self.dropoff_names = self.loc_manager.all_dropoff_names
        
            # Add this line here
            self.all_locations = self.dropoff_names + self.depot_names + [self.gamma]
    
            # Calculate big M based on problem parameters
            self.M = self._calculate_big_M()
    
            # Create Gurobi model with output suppressed
            self.model = Model("TBRD")
            self.model.setParam('OutputFlag', 0)
    
            # Preprocess initial position
            self.preprocess_initial_position()
    
            self._create_variables()
            self._add_constraints()
            self._set_objective()

    def _calculate_big_M(self) -> float:
        """Calculate appropriate Big M value based on problem parameters including handling times"""
        max_travel_time = max(self.instance.truck_travel_times.values(), default=0)
        max_deadline = max(self.instance.deadlines.values(), default=0)
        return max(max_travel_time, max_deadline) * 2
    
    def preprocess_initial_position(self):
        """Handle special cases for initial position"""
        if self.gamma.startswith('R'):  # If γ is a robot depot
            # Track unreachable customers
            self.unreachable_customers = []
            for k in self.instance.C:
                if self.instance.robot_travel_times[(self.gamma, k)] > self.instance.deadlines[k]:
                    self.unreachable_customers.append(k)
        
            # Set initial loading to 0 for depot starts
            self.initial_robots = 0
        else:
            self.initial_robots = self.instance.truck_capacity
            self.unreachable_customers = []

    def _create_variables(self):
        """Create the decision variables for the model"""
        # l[j]: Amount of robots on board at departure from location j
        self.l = self.model.addVars(
            self.dropoff_names + ['gamma'],
            lb=0, name="l"
        )
        
        # s[i,j]: 1 if location j is direct successor of i
        locations = self.dropoff_names + self.depot_names + ['gamma']
        locations_plus = self.dropoff_names + self.depot_names + ['gamma_e']
        self.s = self.model.addVars(
            [(i, j) for i in locations for j in locations_plus if i != j],
            vtype=GRB.BINARY, name="s"
        )
        
        # t[j]: Arrival time at location j
        self.t = self.model.addVars(
            self.dropoff_names + self.depot_names + ['gamma'],
            lb=0, name="t"
        )
        
        # x[j,k]: 1 if customer k is supplied from location j
        self.x = self.model.addVars(
            [(j, k) for j in (self.dropoff_names + self.depot_names + ['gamma'])
             for k in self.instance.C],
            vtype=GRB.BINARY, name="x"
        )
        
        # z[k]: 1 if customer k is supplied late
        self.z = self.model.addVars(
            self.instance.C,
            vtype=GRB.BINARY, name="z"
        )
        
    def _add_visit_frequency_constraints(self):
        """Add constraints for visit frequency limits"""
        K = self.instance.truck_capacity
        C = len(self.instance.C)
    
        # For original drop-off points (considering all duplicates)
        for orig_dropoff in self.loc_manager.original_dropoffs:
            duplicates = self.loc_manager.dropoff_duplicates[orig_dropoff]
            max_visits = math.ceil(C/K) + 1
            for dup in duplicates:
                self.model.addConstr(
                    quicksum(self.s[i,dup] for i in self.all_locations if i != dup) <= 1
                )
    
        # For depots
        max_depot_visits = math.ceil(C/(self.instance.truck_capacity+1)/2)
        for j in self.depot_names:
            self.model.addConstr(
                quicksum(self.s[i,j] for i in self.all_locations if i != j) <= max_depot_visits
            )

    def _add_constraints(self):
        """Add all constraints to the model"""
        # Constraint (2): One-to-one mapping between customers and launch locations
        for k in self.instance.C:
            self.model.addConstr(
                quicksum(self.x[j,k] for j in self.dropoff_names + self.depot_names + ['gamma']) == 1
            )
            
        # Add initial position constraints
        if self.gamma.startswith('R'):
            self.model.addConstr(
                quicksum(self.x[self.gamma,k] for k in self.instance.C) == 0
            )
    
        # Add visit frequency constraints
        self._add_visit_frequency_constraints()
        
        # Constraint (3): Initial loading
        self.model.addConstr(
            self.l['gamma'] <= self.instance.initial_robots - 
            quicksum(self.x['gamma',k] for k in self.instance.C)
        )
        
        # Constraints (4-5): Robot loading after visiting locations
        for i in self.depot_names:
            for j in self.dropoff_names:
                self.model.addConstr(
                    self.l[j] <= self.instance.truck_capacity + 
                    self.M * (1 - self.s[i,j]) -
                    quicksum(self.x[j,k] for k in self.instance.C)
                )
        
        for i in self.dropoff_names + ['gamma']:
            for j in self.dropoff_names:
                if i != j:
                    self.model.addConstr(
                        self.l[j] <= self.l[i] + 
                        self.M * (1 - self.s[i,j]) -
                        quicksum(self.x[j,k] for k in self.instance.C)
                    )
        
        # Constraints (6-7): Arrival times
        self.model.addConstr(self.t['gamma'] == 0)
        
        for i in self.dropoff_names + self.depot_names + ['gamma']:
            for j in self.dropoff_names + self.depot_names:
                if i != j:
                    self.model.addConstr(
                        self.t[j] >= self.t[i] - 
                        self.M * (1 - self.s[i,j]) +
                        self.instance.truck_travel_times.get((i,j), 0)
                    )
        
        # Constraint (8): Late delivery determination
        for j in self.dropoff_names + self.depot_names + ['gamma']:
            for k in self.instance.C:
                self.model.addConstr(
                    self.M * self.z[k] >= 
                    self.t[j] - self.M * (1 - self.x[j,k]) +
                    self.instance.robot_travel_times.get((j,k), 0) - 
                    self.instance.deadlines[k]
                )
        
        # Constraints (9-10): Subtour elimination
        self.model.addConstr(
            quicksum(self.s['gamma',j] for j in 
                    self.dropoff_names + self.depot_names + ['gamma_e']) <= 1
        )
        
        for j in self.dropoff_names + self.depot_names:
            self.model.addConstr(
                quicksum(self.s[j,i] for i in 
                        (self.dropoff_names + self.depot_names + ['gamma_e']) if i != j) ==
                quicksum(self.s[i,j] for i in 
                        (self.dropoff_names + self.depot_names + ['gamma']) if i != j)
            )
        
        # Constraints (11-13): Valid tour constraints
        for j in self.dropoff_names + self.depot_names:
            self.model.addConstr(
                quicksum(self.x[j,k] for k in self.instance.C) <=
                self.M * quicksum(self.s[i,j] for i in 
                                (self.dropoff_names + self.depot_names + ['gamma']) if i != j)
            )
            
        for j in self.dropoff_names:
            self.model.addConstr(
                quicksum(self.x[j,k] for k in self.instance.C) >=
                quicksum(self.s[i,j] for i in 
                        (self.dropoff_names + self.depot_names + ['gamma']) if i != j)
            )
            
        for j in self.depot_names:
            self.model.addConstr(
                quicksum(self.x[j,k] for k in self.instance.C) + 
                quicksum(self.s[j,i] for i in self.dropoff_names) >=
                quicksum(self.s[i,j] for i in 
                        (self.dropoff_names + self.depot_names + ['gamma']) if i != j)
            )

    def _set_objective(self):
        """Set the objective function (1)"""
        self.model.setObjective(
            quicksum(self.z[k] * self.instance.weights[k] for k in self.instance.C),
            GRB.MINIMIZE
        )

    def solve(self, time_limit: int = None) -> int:
        """Solve the model with optional time limit"""
        if time_limit:
            self.model.setParam('TimeLimit', time_limit)
            
        self.model.optimize()
        return self.model.status

    def get_solution(self) -> Dict:
        """Return the solution details if a feasible solution was found"""
        if self.model.SolCount > 0:
            solution = {
                'status': self.model.status,
                'status_text': self.model.status == GRB.OPTIMAL and "Optimal" or "Time Limit Reached",
                'objective_value': self.model.objVal,
                'best_bound': self.model.objBound,
                'gap': self.model.MIPGap,
                'late_deliveries': {k: self.z[k].X for k in self.instance.C},
                'route': self._extract_route(),
                'customer_assignments': self._extract_assignments(),
                'coordinates': self.instance.coordinates,
                'completion_time': self.model.Runtime
            }
            return solution
        return None

    def _extract_route(self) -> List[str]:
        """Extract the truck route from the solution"""
        route = []
        current = self.gamma
        while current != self.gamma_e:
            route.append(current)
            for j in self.dropoff_names + self.depot_names + [self.gamma_e]:
                if j != current and self.s[current,j].X > 0.5:
                    current = j
                    break
        return route

    def _extract_assignments(self) -> Dict[int, str]:
        """Extract customer-to-location assignments from the solution"""
        assignments = {}
        for k in self.instance.C:
            for j in self.dropoff_names + self.depot_names + ['gamma']:
                if self.x[j,k].X > 0.5:
                    assignments[k] = j
        return assignments

def check_feasibility(instance):
    """Analyze why an instance might be infeasible"""
    min_delivery_times = {}
    deadlines_violated = False
    
    # Check delivery time feasibility
    for k in range(len(instance.C)):
        min_time = float('inf')
        customer_pos = instance.coordinates[f'C{k}']
        
        for j in [p for p in instance.coordinates.keys() if p.startswith(('R', 'D', 'gamma'))]:
            service_pos = instance.coordinates[j]
            if ('gamma', j) in instance.truck_travel_times and (j, k) in instance.robot_travel_times:
                total_time = instance.truck_travel_times[('gamma', j)] + instance.robot_travel_times[(j, k)]
                min_time = min(min_time, total_time)
        
        min_delivery_times[k] = min_time
        if min_time > instance.deadlines[k]:
            deadlines_violated = True
    
    return {
        'min_delivery_times': min_delivery_times,
        'deadlines_violated': deadlines_violated,
        'capacity_constrained': len(instance.C) > instance.truck_capacity
    }

def run_experiment(instance_params: Dict, 
                  num_instances: int = 1,
                  time_limit: int = None,
                  save_file: str = None) -> Dict:
    """Run experiments with specified parameters"""
    results = {
        'feasible_instances': [],
        'infeasible_instances': [],
        'summary': {
            'total_instances': num_instances,
            'num_feasible': 0,
            'num_infeasible': 0,
            'num_optimal': 0,
            'avg_solution_time': 0,
            'best_solution_time': float('inf'),
            'worst_solution_time': 0,
            'infeasibility_causes': {
                'deadlines': 0,
                'capacity': 0
            }
        }
    }
    
    print(f"\nRunning {num_instances} instances...")
    generator = TBRDInstanceGenerator(**instance_params)
    
    for i in range(num_instances):
        if (i + 1) % 10 == 0:  # Progress update every 10 instances
            print(f"Processed {i + 1}/{num_instances} instances")
        
        try:
            with timeout(time_limit + 30):  # Add 30 seconds buffer for cleanup
                instance = generator.generate_instance(seed=i)
                model = TBRDModel(instance)
                status = model.solve(time_limit=time_limit)
                solution = model.get_solution()
            
                if solution:
                    # Feasible solution found
                    results['feasible_instances'].append({
                        'instance_id': i,
                        'status': status,
                        'objective_value': solution['objective_value'],
                        'completion_time': solution['completion_time'],
                        'route': solution['route'],
                        'customer_assignments': solution['customer_assignments']
                    })
                    results['summary']['num_feasible'] += 1
                    if status == GRB.OPTIMAL:
                        results['summary']['num_optimal'] += 1
                    
                    # Update timing statistics
                    results['summary']['avg_solution_time'] += solution['completion_time']
                    results['summary']['best_solution_time'] = min(
                        results['summary']['best_solution_time'], 
                        solution['completion_time']
                    )
                    results['summary']['worst_solution_time'] = max(
                        results['summary']['worst_solution_time'], 
                        solution['completion_time']
                    )
                else:
                    # Analyze infeasibility
                    feasibility_check = check_feasibility(instance)
                    results['infeasible_instances'].append({
                        'instance_id': i,
                        'analysis': feasibility_check
                    })
                    results['summary']['num_infeasible'] += 1
                
                    if feasibility_check['deadlines_violated']:
                        results['summary']['infeasibility_causes']['deadlines'] += 1
                    if feasibility_check['capacity_constrained']:
                        results['summary']['infeasibility_causes']['capacity'] += 1
                    
        except TimeoutError:
            print(f"Instance {i} forcibly terminated due to timeout")
            results['infeasible_instances'].append({
                'instance_id': i,
                'analysis': {'timeout': True}
            })
            results['summary']['num_infeasible'] += 1
            continue
    
    # Finalize averages
    if results['summary']['num_feasible'] > 0:
        results['summary']['avg_solution_time'] /= results['summary']['num_feasible']
    
    # Save detailed results if requested
    if save_file:
        import json
        with open(save_file, 'w') as f:
            json.dump(results, f, indent=2)
    
    return results

if __name__ == "__main__":
    instance_params = {
        'w': 2,                # 2 km square area
        'num_customers': 6,    # Number of customers
        'num_dropoff': 6,      # Number of drop-off points
        'num_depots': 4,       # Number of robot depots
        'truck_speed': 30,     # 30 km/h
        'robot_speed': 5,      # 5 km/h
        'truck_capacity': 2,   # Maximum of 2 robots
        'rho_min': 3.0,        # Minimum deadline factor of 3
        'rho_max': 5.0,        # Maximum deadline factor of 5
        'w_min': 1,           # Fixed weight of 1
        'w_max': 1            # Fixed weight of 1
    }
    
    # Run experiments and save detailed results to file
    results = run_experiment(
        instance_params,
        num_instances=25,
        time_limit=200,
        save_file='tbrd_results.json'
    )
    
    # Print summary statistics
    print("\nExperiment Summary:")
    print(f"Total instances: {results['summary']['total_instances']}")
    print(f"Feasible instances: {results['summary']['num_feasible']}")
    print(f"Optimal solutions: {results['summary']['num_optimal']}")
    print(f"Infeasible instances: {results['summary']['num_infeasible']}")
    
    if results['summary']['num_feasible'] > 0:
        print("\nSolution Times:")
        print(f"Average: {results['summary']['avg_solution_time']:.2f} seconds")
        print(f"Best: {results['summary']['best_solution_time']:.2f} seconds")
        print(f"Worst: {results['summary']['worst_solution_time']:.2f} seconds")
    
    print("\nInfeasibility Analysis:")
    print(f"Deadline violations: {results['summary']['infeasibility_causes']['deadlines']}")
    print(f"Capacity constraints: {results['summary']['infeasibility_causes']['capacity']}")
    
    print("\nDetailed results saved to 'tbrd_results.json'")


Running 25 instances...
Processed 10/25 instances
Processed 20/25 instances

Experiment Summary:
Total instances: 25
Feasible instances: 18
Optimal solutions: 18
Infeasible instances: 7

Solution Times:
Average: 0.11 seconds
Best: 0.08 seconds
Worst: 0.17 seconds

Infeasibility Analysis:
Deadline violations: 0
Capacity constraints: 7

Detailed results saved to 'tbrd_results.json'


What i've changed in the above code compared to the one before this one:
- added initial position rules for the truck
- allowed the truck to visit a location multiple times
- fixed handling and service times are now introduced into the travel time calculations